# Function to set up text alert messages
**We should really set up a database to test this...**

For 10 minute averages of all sensors (using the ATM PurpleAir estimations)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and texts people who are interested in the sensors if they are above a threshold

## Import Packages

In [3]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

# import psycopg2
# from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

# Get CWD

cwd = os.getcwd()

## Definitions

In [12]:
# This is my personal API key... Please use responsibly! 51592903-B445-11ED-B6F4-42010A800007

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


In [13]:
# Load the necessary data

datapath = os.path.join(cwd, '..', '..', 'Data')

sensor_info = gpd.read_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

sensor_ids = sensor_info.sensor_index.unique().astype(int) # The sensor indices

In [14]:
# Set the Spike threshold

spike_threshold = 28 # Micgrograms per meter cubed

## Check for Spikes

In [10]:
# Check for spikes

# Please see Scripts/Get_spikes_df.py

exec(open(os.path.join('..', '..', 'Scripts', 'Get_spikes_df.py')).read())

help(Get_spikes_df)

Help on function Get_spikes_df in module __main__:

Get_spikes_df(api, sensor_ids, spike_threshold)
    This function queries the PurpleAir API for sensors in the list of sensor_ids for readings over a spike threshold. 
    It will return a pandas dataframe with columns sensor_index (integer) and pm25 (float) as well as a runtime (datetime)
    
    Inputs:
    
    api = string of PurpleAir API api_read_key
    sensor_ids = list of integers of purpleair sensor ids to query
    spike_threshold = float of spike value threshold (keep values >=)
    
    Outputs:
    
    spikes_df = Pandas DataFrame with fields sensor_index (integer) and pm25 (float)
    runtime = datetime object when query was run



In [16]:
spikes_df, runtime = Get_spikes_df(api, sensor_ids, spike_threshold)

# Update Current Alerts

In [27]:
# Load active alerts (this should be a database thing...)

active_alerts = pd.read_csv(os.path.join(datapath, 'Active_Alerts_Acute_PurpleAir.csv'))

In [28]:
# Check if any alerts ended

current_spiked_sensors = set(spikes_df.sensor_index)

old_spiked_sensors = set(active_alerts.sensor_index.astype(int)) # This should be a database thing

ended_spiked_sensors = old_spiked_sensors - current_spiked_sensors

In [40]:
# If there are ended alerts, we should:

# 1) Add to archived alerts
# 2) Message people it's over
# 3) Remove from active alerts

if len(ended_spiked_sensors) > 0:

    # 1) Add them to alert archive

    archived_alerts = pd.read_csv(os.path.join(datapath, 'Archived_Alerts_Acute_PurpleAir.csv'))
    
    done_alerts = active_alerts[active_alerts.sensor_index.isin(ended_spiked_sensors)].copy()

    durations = ((runtime - pd.to_datetime(done_alerts.start_time)).dt.total_seconds()/60).astype(int)
    # ^ This subtracts start_times of the done alerts from runtime, gets total seconds, divides by 60
    # (to get minutes), and rounds to nearest integer
    
    done_alerts['duration'] = durations

    # Concatenate and save (this really should be a submission using SQL...
    pd.concat([archived_alerts, done_alerts]).to_csv(os.path.join(datapath, 'Archived_Alerts_Acute_PurpleAir.csv'))


    # 2) Text people the spike is over
    # NOT DONE HERE

    # 3) Remove from current alerts (should be a database thing...)
    
    current_alerts = current_alerts[~current_alerts.sensor_index.isin(ended_spiked_sensors)]

In [45]:
current_alerts.loc[len(current_alerts)-1, :]

sensor_index                      168327.0
start_time      2023-08-21 09:54:47.240388
max_reading                           32.7
Name: 44, dtype: object

In [49]:
# Check if there is currently an alert out on this
# Again, should be a database thing...

for _, spike in spikes_df.iterrows():

    sensor_index = int(spike.sensor_index)
    reading = spike.pm25

    # Is there already an alert for this?
    
    if sensor_index in list(current_alerts.sensor_index.astype(int)): # If yes

         # Check if reading is higher than previous max and update
        
        alert = current_alerts[current_alerts.sensor_index == sensor_index].iloc[0] # Find the alert
        
        if reading > alert.max_reading:
            
            current_alerts.loc[alert.name, 'max_reading'] = reading # Replace max_reading

    else:

        # Start an alert

        new_alert = [sensor_index, runtime, reading]

        current_alerts.loc[len(current_alerts), :] = new_alert # Should be SQL

        # Text everyone...
        # NOT DONE
        # message = create_message()

        message = f'''SPIKE ALERT! {sensor_index} is reading at {reading} micrograms/meter^3.
        You are receiving this text because you signed up for SpikeAlerts. 
        Please reply with STOP to be removed from this list.'''
        
        print(message)
        
        # print(message)

In [50]:
current_alerts.to_csv(os.path.join(datapath, 'Active_Alerts_Acute_PurpleAir.csv'),
                      index=False)